In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression, f_classif, SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [6]:
# Загрузка данных (предположим, что данные в CSV)
df = pd.read_csv("./datasets/train/train.csv")  # Замените на ваш путь к файлу

# Удаляем индекс, так как он не несёт информации для модели
X = df.drop(['index', 'target'], axis=1)  # Все фичи (feature_1 ... feature_1366)
y = df['target']  # Целевая переменная

X, y

(        feature_0  feature_1  ...  feature_1365  feature_1366
 0        0.128367        1.0  ...             1             0
 1        0.258685        1.0  ...             0             0
 2        0.407738        1.0  ...             0             0
 3        0.439134        1.0  ...             0             0
 4        0.590481        1.0  ...             0             0
 ...           ...        ...  ...           ...           ...
 177853   0.660324        1.0  ...             1             1
 177854   0.222214        0.0  ...             0             0
 177855   0.545399        0.0  ...             0             0
 177856   0.393150        1.0  ...             0             1
 177857   0.395272        1.0  ...             0             0
 
 [177858 rows x 1367 columns],
 0         0
 1         0
 2         0
 3         0
 4         0
          ..
 177853    0
 177854    0
 177855    0
 177856    0
 177857    0
 Name: target, Length: 177858, dtype: int64)

In [ ]:
# 1. Отбор признаков: удаляем низковариативные (константные и почти константные)
# Сделаем pipeline для более эффективного отбора признаков(их сокращения)
pipe = Pipeline([
    ('selector', VarianceThreshold(threshold=0.01)),
    ('kbest', SelectKBest(f_regression, k=1000)),
    ('lasso_selection', SelectFromModel(LassoCV(cv=5), max_features=500))
])

# 3. Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_selected = pipe.fit_transform(X_train, y_train)

print(f"Исходное число признаков: {X.shape[1]}")
print(f"Число признаков после отбора: {X_selected.shape[1]}")

In [ ]:
# 2. Масштабирование данных (важно для линейной регрессии)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)

# 4. Создание и обучение модели
model = LinearRegression()
model.fit(X_train, y_train)

# 5. Предсказание и оценка качества
y_pred = model.predict(X_test)

# Метрики
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\nMSE: {mse:.4f}")
print(f"R2 Score: {r2:.4f}")

# (Опционально) Вывод коэффициентов модели
if X_selected.shape[1] <= 20:  # Не выводить для слишком большого числа фич
    print("\nКоэффициенты модели:")
    for feature, coef in zip(X.columns[selector.get_support()], model.coef_):
        print(f"{feature}: {coef:.4f}")
else:
    print(f"\nМодель имеет {X_selected.shape[1]} коэффициентов (слишком много для вывода).")